In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sktime.classification.compose import ColumnEnsembleClassifier
from sktime.classification.dictionary_based import BOSSEnsemble
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.shapelet_based import MrSEQLClassifier
from sktime.datasets import load_basic_motions
from sktime.transformations.panel.compose import ColumnConcatenator

/home/geo/.local/lib/python3.8/site-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [3]:
X, y = load_basic_motions(return_X_y=True)

#X = np.load("./Data/Preprocessed_Data/data.npy", allow_pickle=True)
#y = np.load("./Data/Preprocessed_Data/labels.npy", allow_pickle=True)
#X = pd.DataFrame(X)
#y = pd.DataFrame(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(82, 1) (82, 1) (28, 1) (28, 1)


In [5]:
#  multivariate input data
X_train.head()

,0
15,"[[-1.2246713638305664, -1.4257898330688477, 9...."
44,"[[-2.114144325256348, -2.201535224914551, 9.31..."
89,"[[-1.0391149520874023, -3.010800361633301, 9.0..."
90,"[[-1.5670518875122068, 0.8104610443115234, 9.7..."
9,"[[-1.300090789794922, 2.8874950408935547, 9.20..."


In [6]:
# multi-class target variable
np.unique(y_train)

array([101, 102, 103, 104, 105, 106, 107, 115])

In [7]:
steps = [
    ("concatenate", ColumnConcatenator()),
    ("classify", TimeSeriesForestClassifier(n_estimators=100)),
]
clf = Pipeline(steps)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

/home/geo/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


ValueError: Tabularization failed, it's possible that not all series were of equal length

In [6]:
clf = ColumnEnsembleClassifier(
    estimators=[
        ("TSF0", TimeSeriesForestClassifier(n_estimators=100), [0]),
        ("BOSSEnsemble3", BOSSEnsemble(max_ensemble_size=5), [3]),
    ]
)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.95

In [7]:
clf = MrSEQLClassifier()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0